# Learning Scikit-learn: Machine Learning in Python

## WEEK 3: Supervised Learning - Text Classification with Naïve Bayes

One of the most successful applications of Naive Bayes has been within the field
of Natural Language Processing (NLP). NLP is a field that has been much related
to machine learning, since many of its problems can be formulated as a classification task. Usually, NLP problems have important amounts of tagged data in the form of text documents. This data can be used as a training dataset for machine
learning algorithms.
In this section, we will use Naive Bayes for text classification; we will have a set of text documents with their corresponding categories, and we will train a Naive Bayes algorithm to learn to predict the categories of new unseen instances. This simple task has many practical applications; probably the most known and widely used one is spam filtering. In this section we will try to classify newsgroup messages using a dataset that can be retrieved from within scikit-learn. This dataset consists of around 19,000 newsgroup messages from 20 different topics ranging from politics and religion to sports and science.

Start by importing numpy, scikit-learn, and pyplot, the Python libraries we will be using in this chapter. Show the versions we will be using (in case you have problems running the notebooks).

In [1]:
%matplotlib inline
import IPython
import sklearn as sk
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

DATA_PATH = '/data/MLSem/Ex3/'
print ('IPython version:', IPython.__version__)
print ('numpy version:', np.__version__)
print ('scikit-learn version:', sk.__version__)
print ('matplotlib version:', matplotlib.__version__)

IPython version: 4.2.0
numpy version: 1.11.0
scikit-learn version: 0.17.1
matplotlib version: 1.5.1


Import the newsgroup Dataset, and explore its structure and data (this could take some time, especially if sklearn has to download the 14MB dataset from the Internet)

In [2]:
from sklearn.datasets import fetch_20newsgroups
news = fetch_20newsgroups(data_home=DATA_PATH, subset='all')

Let's explore the dataset structure:

In [3]:
news.keys()

dict_keys(['DESCR', 'filenames', 'target', 'target_names', 'data', 'description'])

If we look at the properties of the dataset, we will find that we have the usual ones: DESCR, data, target, and target_names. The difference now is that data holds a list of text contents, instead of a numpy matrix:

In [4]:
print (type(news.data), type(news.target), type(news.target_names))
print (news.target_names)
print (len(news.data))
print (len(news.target))

<class 'list'> <class 'numpy.ndarray'> <class 'list'>
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
18846
18846


If you look at, say, the first instance, you will see the content of a newsgroup message, and you can get its corresponding category:

In [5]:
print (news.data[0])
print (news.target[0], news.target_names[news.target[0]])

From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>
Subject: Pens fans reactions
Organization: Post Office, Carnegie Mellon, Pittsburgh, PA
Lines: 12
NNTP-Posting-Host: po4.andrew.cmu.edu



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!


10 rec.sport.hockey


Let's build the training and testing datasets:

In [6]:
SPLIT_PERC = 0.75
split_size = int(len(news.data)*SPLIT_PERC)
X_train = news.data[:split_size]
X_test = news.data[split_size:]
y_train = news.target[:split_size]
y_test = news.target[split_size:]

This function will serve to perform and evaluate a cross validation:

In [7]:
from sklearn.cross_validation import cross_val_score, KFold
from scipy.stats import sem

def mean_score(scores):
    """Print the empirical mean score and standard error of the mean."""
    return ("Mean score: {0:.3f} (+/-{1:.3f})").format(
        np.mean(scores), sem(scores))

def evaluate_cross_validation(clf, X, y, K):
    # create a k-fold croos validation iterator of k=5 folds
    cv = KFold(len(y), K, shuffle=True, random_state=0)
    # by default the score used is the one returned by score method of the estimator (accuracy)
    scores = cross_val_score(clf, X, y, cv=cv)
    print (scores)
    print (mean_score(scores))

In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer, CountVectorizer

clf_1 = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB()),
])
clf_2 = Pipeline([
    ('vect', HashingVectorizer(non_negative=True)),
    ('clf', MultinomialNB()),
])
clf_3 = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultinomialNB()),
])

Our machine learning algorithms can work only on numeric data, so our next step will be to convert our text-based dataset to a numeric dataset. Currently we only have one feature, the text content of the message; we need some function that transforms a text into a meaningful set of numeric features. Intuitively one could try to look at which are the words (or more precisely, tokens, including numbers or punctuation signs) that are used in each of the text categories, and try to characterize each category with the frequency distribution of each of those words. The sklearn. feature_extraction.text module has some useful utilities to build numeric feature vectors from text documents.

If you look inside the sklearn.feature_extraction.text module, you will find three different classes that can transform text into numeric features: CountVectorizer, HashingVectorizer, and TfidfVectorizer. The difference between them resides in the calculations they perform to obtain the numeric features. CountVectorizer basically creates a dictionary of words from the text corpus. Then, each instance is converted to a vector of numeric features where each element will be the count of the number of times a particular word appears in the document. HashingVectorizer, instead of constricting and maintaining the dictionary in memory, implements a hashing function that maps tokens into feature indexes, and then computes the count as in CountVectorizer. TfidfVectorizer works like the CountVectorizer, but with a more advanced calculation called Term Frequency Inverse Document Frequency (TF-IDF). This is a statistic for measuring the importance of a word in a document or corpus. Intuitively, it looks for words that are more frequent in the current document, compared with their frequency in the whole corpus of documents. You can see this as a way to normalize the results and avoid words that are too frequent, and thus not useful to characterize the instances.

We will create a Naive Bayes classifier that is composed of a feature vectorizer and the actual Bayes classifier. We will use the MultinomialNB class from the sklearn.naive_bayes module (following code may take couple of minutes to complete! Be patient!).


In [9]:
clfs = [clf_1, clf_2, clf_3]
for clf in clfs:
    evaluate_cross_validation(clf, news.data, news.target, 5)


[ 0.85782493  0.85725657  0.84664367  0.85911382  0.8458477 ]
Mean score: 0.853 (+/-0.003)
[ 0.75543767  0.77659857  0.77049615  0.78508888  0.76200584]
Mean score: 0.770 (+/-0.005)
[ 0.84482759  0.85990979  0.84558238  0.85990979  0.84213319]
Mean score: 0.850 (+/-0.004)


We will keep the TF-IDF vectorizer but use a different regular expression to pefrom tokenization. The default regular expression: ```r"\b\w\w+\b"``` considers alphanumeric characters and the underscore. Perhaps also considering the slash and the dot could improve the tokenization, and begin considering tokens as Wi-Fi and site.com. The new regular expression could be: ```r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b"```. If you have queries about how to define regular expressions, please refer to the Python re module documentation. Let's try our new classifier:

In [10]:
clf_4 = Pipeline([
    ('vect', TfidfVectorizer(token_pattern=r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b")),
    ('clf', MultinomialNB()),
])

In [11]:
evaluate_cross_validation(clf_4, news.data, news.target, 5)

[ 0.86100796  0.8718493   0.86203237  0.87291059  0.8588485 ]
Mean score: 0.865 (+/-0.003)


Another parameter that we can use is stop_words: this argument allows us to pass a list of words we do not want to take into account, such as too frequent words, or words we do not a priori expect to provide information about the particular topic. Let's try to improve performance filtering the stop words:

In [12]:
def get_stop_words():
    result = set()
    for line in open(DATA_PATH + 'stopwords_en.txt', 'r').readlines():
        result.add(line.strip())
    return result

In [13]:
stop_words = get_stop_words()
print (stop_words)


{'though', 'except', 'whereas', 'in', 'among', 'any', 'becoming', 'co', 'much', 'anyhow', 'elsewhere', 'give', 'one', 'put', 'that', 'our', 'only', 'via', 'had', 'bottom', 'his', 'find', 'meanwhile', 'yourself', 'whom', 'thin', 'across', 'mill', 'wherein', 'due', 'too', 'against', 'hasnt', 'how', 'themselves', 'towards', 'along', 'namely', 'herself', 'first', 'its', 'become', 'has', 'not', 're', 'made', 'none', 'can', 'several', 'your', 'inc', 'detail', 'my', 'almost', 'are', 'fifteen', 'both', 'he', 'hereupon', 'cannot', 'perhaps', 'least', 'serious', 'system', 'each', 'of', 'off', 'twenty', 'never', 'through', 'two', 'me', 'ten', 'would', 'been', 'etc', 'no', 'until', 'found', 'into', 'itself', 'ever', 'very', 'may', 'again', 'beside', 'done', 'seems', 'get', 'therefore', 'move', 'former', 'whereupon', 'down', 'forty', 'show', 'whole', 'i', 'third', 'something', 'should', 'nevertheless', 'this', 'himself', 'during', 'now', 'however', 'seem', 'than', 'from', 'always', 'if', 'go', 'wha

In [14]:
clf_5 = Pipeline([
    ('vect', TfidfVectorizer(
                stop_words=stop_words,
                token_pattern=r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b",    
    )),
    ('clf', MultinomialNB()),
])

In [15]:
evaluate_cross_validation(clf_5, news.data, news.target, 5)

[ 0.88116711  0.89519767  0.88325816  0.89227912  0.88113558]
Mean score: 0.887 (+/-0.003)


Try to improve by adjusting the alpha parameter on the MultinomialNB classifier:

In [16]:
clf_7 = Pipeline([
    ('vect', TfidfVectorizer(
                stop_words=stop_words,
                token_pattern=r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b",         
    )),
    ('clf', MultinomialNB(alpha=0.01)),
])

In [17]:
evaluate_cross_validation(clf_7, news.data, news.target, 5)

[ 0.9204244   0.91960732  0.91828071  0.92677103  0.91854603]
Mean score: 0.921 (+/-0.002)


The results had an important boost from 0.89 to 0.92, pretty good. At this point, we could continue doing trials by using different values of alpha or doing new modifications of the vectorizer. In Week 4, Advanced Features, we will show you practical utilities to try many different configurations and keep the best one.

If we decide that we have made enough improvements in our model, we are ready to evaluate its performance on the testing set.

In [18]:
from sklearn import metrics

def train_and_evaluate(clf, X_train, X_test, y_train, y_test):
    
    clf.fit(X_train, y_train)
    
    print ("Accuracy on training set:")
    print (clf.score(X_train, y_train))
    print ("Accuracy on testing set:")
    print (clf.score(X_test, y_test))
    
    y_pred = clf.predict(X_test)
    
    print ("Classification Report:")
    print (metrics.classification_report(y_test, y_pred))
    print ("Confusion Matrix:")
    print (metrics.confusion_matrix(y_test, y_pred))

In [19]:
train_and_evaluate(clf_7, X_train, X_test, y_train, y_test)

Accuracy on training set:
0.996957690675
Accuracy on testing set:
0.917869269949
Classification Report:
             precision    recall  f1-score   support

          0       0.95      0.88      0.91       216
          1       0.85      0.85      0.85       246
          2       0.91      0.84      0.87       274
          3       0.81      0.86      0.83       235
          4       0.88      0.90      0.89       231
          5       0.89      0.91      0.90       225
          6       0.88      0.80      0.84       248
          7       0.92      0.93      0.93       275
          8       0.96      0.98      0.97       226
          9       0.97      0.94      0.96       250
         10       0.97      1.00      0.98       257
         11       0.97      0.97      0.97       261
         12       0.90      0.91      0.91       216
         13       0.94      0.95      0.95       257
         14       0.94      0.97      0.95       246
         15       0.90      0.96      0.93     

As we can see, we obtained very good results, and as we would expect, the accuracy in the training set is quite better than in the testing set. We may expect, in new unseen instances, an accuracy of around 0.91.

If we look inside the vectorizer, we can see which tokens have been used to create our dictionary (these are the first 20):

In [20]:
clf_7.named_steps['vect'].get_feature_names()[:20]

['...and',
 '0-.66d8wt',
 '0-04g55',
 '0-100mph',
 '0-13-117441-x--or',
 '0-3mb',
 '0-40mb',
 '0-40volts',
 '0-5mb',
 '0-60mph',
 '0-8.3mb',
 '0-a00138',
 '0-byte',
 '0-defects',
 '0-e8',
 '0-for-4',
 '0-hc',
 '0-ii',
 '0-uw',
 '0-uw0']

In [21]:
print (len(clf_7.named_steps['vect'].get_feature_names()))

145767


---